In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from scipy import stats
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , r2_score
%matplotlib inline
# 한글깨짐 방지 실행
!python han-font.py
exec(open('han-font.py').read())

In [2]:
diabetes = load_diabetes()
diabetes

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990842, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06832974, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286377, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04687948,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452837, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00421986,  0.00306441]]),
 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59

In [3]:
diabetes_df = pd.DataFrame(diabetes.data, columns = diabetes.feature_names)
diabetes_df["diabetes value"]=diabetes.target
diabetes_df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,diabetes value
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [4]:
##훈련용, 테스트용 자료 분류 : fixed
x_data=diabetes_df.iloc[:, 0:10]
y_data=diabetes_df["diabetes value"]
num_of_train = int(len(x_data) * 0.8) # 데이터의 전체 길이의 80%에 해당하는 길이값을 구한다.
num_of_test = int(len(x_data) - num_of_train) # 전체 길이에서 80%에 해당하는 길이를 뺀다.
print('훈련 데이터의 크기 :',num_of_train)
print('테스트 데이터의 크기 :',num_of_test)

훈련 데이터의 크기 : 353
테스트 데이터의 크기 : 89


In [5]:
x_test = x_data[num_of_train:] # 전체 데이터 중에서 20%만큼 뒤의 데이터 저장
y_test = y_data[num_of_train:] # 전체 데이터 중에서 20%만큼 뒤의 데이터 저장
x_train = x_data[:num_of_train] # 전체 데이터 중에서 80%만큼 앞의 데이터 저장
y_train = y_data[:num_of_train] # 전체 데이터 중에서 80%만큼 앞의 데이터 저장

In [6]:
# x_train과 x_test를 StandardScaler를 이용해 정규화
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_train=pd.DataFrame(x_train, columns=x_data.columns)
x_test=pd.DataFrame(x_test, columns=x_data.columns)
x_train.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.767844,1.067424,1.347413,0.454905,-0.935888,-0.741500,-0.900815,-0.051951,0.440448,-0.359531
1,-0.057153,-0.936835,-1.100358,-0.560668,-0.178229,-0.410729,1.521083,-0.808181,-1.457857,-1.937199
2,1.742841,1.067424,0.974420,-0.125422,-0.965029,-0.728269,-0.673762,-0.051951,0.073759,-0.534828
3,-1.857146,-0.936835,-0.237810,-0.778291,0.258882,0.522048,-0.749446,0.704279,0.500333,-0.184235
4,0.092847,-0.936835,-0.773988,0.454905,0.084038,0.323585,0.158766,-0.051951,-0.676096,-0.973069


In [7]:
#1. 다중회귀모형 분석
# 다중회귀모형분석(full-model)
diabetes_train=x_train.join(y_train)
fit1=smf.ols('y_train~x_train', diabetes_train).fit()
print(fit1.summary())

                            OLS Regression Results                            
Dep. Variable:                y_train   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.495
Method:                 Least Squares   F-statistic:                     35.44
Date:                Tue, 06 Dec 2022   Prob (F-statistic):           4.02e-47
Time:                        03:47:49   Log-Likelihood:                -1905.0
No. Observations:                 353   AIC:                             3832.
Df Residuals:                     342   BIC:                             3874.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    151.4788      2.887     52.471      0.0

In [8]:
# Linear Regression OLS로 학습/예측평가 수행. 
reg= LinearRegression()
reg.fit(x_train ,y_train)
y_preds= reg.predict(x_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)

In [9]:
# MSE, RMSE
print('MSE : {0:.3f} , RMSE : {1:.3F}'.format(mse , rmse))

MSE : 2929.887 , RMSE : 54.128


In [10]:
#2.주성분분석
# PCA모듈설치 및 PCA분석+주성분의 회귀계수를 데이터프레임으로 구성
import statsmodels.formula.api as smf
from sklearn.decomposition import PCA
pca = PCA(n_components=4)   # 주성분을 몇개로 할지 결정
printcipalComponents = pca.fit_transform(x_train)
principalDf = pd.DataFrame(data=printcipalComponents, columns = 
                           ['pca1', 'pca2', 'pca3', 'pca4'])

In [11]:
# 각 주성분의 설명력
pca.explained_variance_ratio_

array([0.40840746, 0.14966528, 0.11804921, 0.09612724])

In [12]:
sum(pca.explained_variance_ratio_)   # 누적설명력

0.772249187767218

In [13]:
# 주성분분석 : 고유벡터(주성분 식의 계수값)
pca.components=pd.DataFrame(pca.components_)
pca.components

,0,1,2,3,4,5,6,7,8,9
0,0.210174,0.168510,0.316500,0.269375,0.346246,0.357016,-0.273829,0.424726,0.376457,0.332046
1,-0.138300,0.438194,0.087941,0.025278,-0.540536,-0.378793,-0.563160,0.153070,0.005876,0.064881
2,0.502106,0.059905,0.138816,0.535366,-0.166151,-0.351024,0.297858,-0.357257,0.071758,0.257077
3,-0.383641,-0.668344,0.476510,0.063620,-0.119878,-0.237042,-0.067010,-0.019469,0.305431,0.079430


In [14]:
# 주성분의 데이터프레임 생성
pca_xtrain=pd.DataFrame()
pca_xtrain['pca1']=principalDf['pca1']
pca_xtrain['pca2']=principalDf['pca2']
pca_xtrain['pca3']=principalDf['pca3']
pca_xtrain['pca4']=principalDf['pca4']
pca_xtrain.head()

,pca1,pca2,pca3,pca4
0,0.572508,1.756905,0.985274,0.118318
1,-2.829351,-1.376205,-0.224767,-0.478562
2,0.239260,1.443937,1.108810,-0.611166
3,0.074432,-0.011093,-2.150851,1.195681
4,-0.759321,-0.812388,-0.233773,-0.129561


In [15]:
pca_train=pca_xtrain.join(y_train)
fit2=smf.ols('y_train~pca1+pca2+pca3+pca4', pca_train).fit()
print(fit2.summary())

                            OLS Regression Results                            
Dep. Variable:                y_train   R-squared:                       0.498
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                     86.46
Date:                Tue, 06 Dec 2022   Prob (F-statistic):           6.27e-51
Time:                        03:47:53   Log-Likelihood:                -1908.7
No. Observations:                 353   AIC:                             3827.
Df Residuals:                     348   BIC:                             3847.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    151.4788      2.892     52.376      0.0

In [16]:
# test-set을 예측하기 위해 PCA점수로 변환 후 예측
arr1=pca.components_         
arr2=np.array(x_test.T)      
result=arr1@arr2             

In [17]:
# 주성분의 이름부여 및 원자료에 주성분점수 추가
cols = ["pca1", "pca2", "pca3", "pca4"]
pca_xtest = pd.DataFrame(result.T, columns=cols)   # result.T 12*5
pca_xtest.head()

,pca1,pca2,pca3,pca4
0,1.008926,-1.335976,-1.461661,1.078734
1,3.321943,-1.132319,-0.650724,-0.156449
2,-0.685136,1.447582,-0.339103,-0.156566
3,-0.842130,-0.093892,-0.770310,-1.448587
4,0.089160,1.308508,-0.259013,0.766078


In [18]:
# Linear Regression OLS로 학습/예측평가 수행. 
from sklearn.metrics import mean_squared_error , r2_score
y_preds= fit2.predict(pca_xtest)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)
# MSE, RMSE, R^2, 설명분산점수(적합모형의 편향성 평가 : 0에 가까워야)
print('MSE : {0:.3f} , RMSE : {1:.3F}'.format(mse , rmse))

MSE : 3104.153 , RMSE : 55.715


In [19]:
#3. 신경망(예측) 
#3.1 회귀 모델 생성 - reiu 활성함수
import tensorflow as tf
model_reiu = tf.keras.Sequential([
    tf.keras.layers.Dense(units=52, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(units=39, activation='relu'),
    tf.keras.layers.Dense(units=26, activation='relu'),
    tf.keras.layers.Dense(units=1)
])
model_reiu.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01), loss='mse')
model_reiu.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 52)                572       
                                                                 
 dense_1 (Dense)             (None, 39)                2067      
                                                                 
 dense_2 (Dense)             (None, 26)                1040      
                                                                 
 dense_3 (Dense)             (None, 1)                 27        
                                                                 
Total params: 3,706
Trainable params: 3,706
Non-trainable params: 0
_________________________________________________________________


In [27]:
# 회귀 모델 학습 : epochs~신경망을 전체 한번 학습하여 나온 결과
# batch_size~train-set의 80%의 자료를 32개씩 소그룹(sub-group)으로 나누어 빠르게 학습
model_reiu.fit(x_train, y_train, epochs=3000, batch_size=32, validation_split=0.2)

Epoch 1/3000
9/9 [==============================] - 0s 9ms/step - loss: 2514.5376 - val_loss: 2814.7693
Epoch 2/3000
9/9 [==============================] - 0s 5ms/step - loss: 2500.8113 - val_loss: 2803.7207
Epoch 3/3000
9/9 [==============================] - 0s 4ms/step - loss: 2500.0786 - val_loss: 2801.6370
Epoch 4/3000
9/9 [==============================] - 0s 5ms/step - loss: 2506.2617 - val_loss: 2789.2986
Epoch 5/3000
9/9 [==============================] - 0s 5ms/step - loss: 2496.0974 - val_loss: 2792.3962
Epoch 6/3000
9/9 [==============================] - 0s 5ms/step - loss: 2489.9417 - val_loss: 2798.6333
Epoch 7/3000
9/9 [==============================] - 0s 5ms/step - loss: 2490.7078 - val_loss: 2805.4053
Epoch 8/3000
9/9 [==============================] - 0s 5ms/step - loss: 2490.3523 - val_loss: 2797.8948
Epoch 9/3000
9/9 [==============================] - 0s 5ms/step - loss: 2486.2500 - val_loss: 2800.9587
Epoch 10/3000
9/9 [==============================] - 0s 5ms/step

9/9 [==============================] - 0s 4ms/step - loss: 2236.9067 - val_loss: 2831.3574
Epoch 80/3000
9/9 [==============================] - 0s 5ms/step - loss: 2252.2576 - val_loss: 2814.0215
Epoch 81/3000
9/9 [==============================] - 0s 4ms/step - loss: 2233.3140 - val_loss: 2823.1189
Epoch 82/3000
9/9 [==============================] - 0s 5ms/step - loss: 2224.0688 - val_loss: 2828.0659
Epoch 83/3000
9/9 [==============================] - 0s 5ms/step - loss: 2220.0459 - val_loss: 2831.5842
Epoch 84/3000
9/9 [==============================] - 0s 5ms/step - loss: 2218.6201 - val_loss: 2825.2288
Epoch 85/3000
9/9 [==============================] - 0s 5ms/step - loss: 2208.4019 - val_loss: 2842.5051
Epoch 86/3000
9/9 [==============================] - 0s 5ms/step - loss: 2227.5918 - val_loss: 2856.5691
Epoch 87/3000
9/9 [==============================] - 0s 5ms/step - loss: 2217.7961 - val_loss: 2838.5168
Epoch 88/3000
9/9 [==============================] - 0s 5ms/step - lo

9/9 [==============================] - 0s 5ms/step - loss: 1825.9186 - val_loss: 2878.8774
Epoch 157/3000
9/9 [==============================] - 0s 5ms/step - loss: 1816.7212 - val_loss: 2895.1641
Epoch 158/3000
9/9 [==============================] - 0s 5ms/step - loss: 1814.8312 - val_loss: 2887.5127
Epoch 159/3000
9/9 [==============================] - 0s 5ms/step - loss: 1816.9165 - val_loss: 2886.0269
Epoch 160/3000
9/9 [==============================] - 0s 4ms/step - loss: 1808.3210 - val_loss: 2874.6475
Epoch 161/3000
9/9 [==============================] - 0s 5ms/step - loss: 1788.0499 - val_loss: 2877.8518
Epoch 162/3000
9/9 [==============================] - 0s 5ms/step - loss: 1795.7307 - val_loss: 2889.0337
Epoch 163/3000
9/9 [==============================] - 0s 5ms/step - loss: 1780.4442 - val_loss: 2893.6252
Epoch 164/3000
9/9 [==============================] - 0s 5ms/step - loss: 1801.7565 - val_loss: 2879.3582
Epoch 165/3000
9/9 [==============================] - 0s 5ms/

9/9 [==============================] - 0s 5ms/step - loss: 1399.2157 - val_loss: 3129.6829
Epoch 234/3000
9/9 [==============================] - 0s 5ms/step - loss: 1389.5785 - val_loss: 3140.5457
Epoch 235/3000
9/9 [==============================] - 0s 4ms/step - loss: 1391.0740 - val_loss: 3145.2041
Epoch 236/3000
9/9 [==============================] - 0s 5ms/step - loss: 1389.5350 - val_loss: 3128.0300
Epoch 237/3000
9/9 [==============================] - 0s 5ms/step - loss: 1374.8632 - val_loss: 3152.7178
Epoch 238/3000
9/9 [==============================] - 0s 5ms/step - loss: 1370.0375 - val_loss: 3145.3164
Epoch 239/3000
9/9 [==============================] - 0s 5ms/step - loss: 1363.8131 - val_loss: 3161.1123
Epoch 240/3000
9/9 [==============================] - 0s 5ms/step - loss: 1358.6458 - val_loss: 3171.2031
Epoch 241/3000
9/9 [==============================] - 0s 5ms/step - loss: 1371.9167 - val_loss: 3178.0044
Epoch 242/3000
9/9 [==============================] - 0s 5ms/

9/9 [==============================] - 0s 5ms/step - loss: 1001.8423 - val_loss: 3634.4036
Epoch 311/3000
9/9 [==============================] - 0s 5ms/step - loss: 1010.2943 - val_loss: 3651.2988
Epoch 312/3000
9/9 [==============================] - 0s 5ms/step - loss: 998.5647 - val_loss: 3630.1646
Epoch 313/3000
9/9 [==============================] - 0s 5ms/step - loss: 1007.4306 - val_loss: 3657.4365
Epoch 314/3000
9/9 [==============================] - 0s 5ms/step - loss: 1003.2300 - val_loss: 3647.9932
Epoch 315/3000
9/9 [==============================] - 0s 5ms/step - loss: 986.3107 - val_loss: 3652.0859
Epoch 316/3000
9/9 [==============================] - 0s 5ms/step - loss: 977.0333 - val_loss: 3678.6431
Epoch 317/3000
9/9 [==============================] - 0s 5ms/step - loss: 976.6258 - val_loss: 3670.8635
Epoch 318/3000
9/9 [==============================] - 0s 5ms/step - loss: 989.4767 - val_loss: 3654.1616
Epoch 319/3000
9/9 [==============================] - 0s 5ms/step 

...

Epoch 2661/3000
9/9 [==============================] - 0s 5ms/step - loss: 4.4260 - val_loss: 7530.6240
Epoch 2662/3000
9/9 [==============================] - 0s 5ms/step - loss: 4.2929 - val_loss: 7431.6406
Epoch 2663/3000
9/9 [==============================] - 0s 5ms/step - loss: 4.8466 - val_loss: 7607.4224
Epoch 2664/3000
9/9 [==============================] - 0s 5ms/step - loss: 2.9373 - val_loss: 7531.3135
Epoch 2665/3000
9/9 [==============================] - 0s 5ms/step - loss: 3.0698 - val_loss: 7469.3916
Epoch 2666/3000
9/9 [==============================] - 0s 5ms/step - loss: 2.3882 - val_loss: 7541.3105
Epoch 2667/3000
9/9 [==============================] - 0s 5ms/step - loss: 1.5171 - val_loss: 7577.2842
Epoch 2668/3000
9/9 [==============================] - 0s 5ms/step - loss: 1.1757 - val_loss: 7528.7095
Epoch 2669/3000
9/9 [==============================] - 0s 5ms/step - loss: 1.4050 - val_loss: 7523.1719
Epoch 2670/3000
9/9 [==============================] - 0s 5ms/st

9/9 [==============================] - 0s 4ms/step - loss: 3.4754 - val_loss: 7490.3311
Epoch 2740/3000
9/9 [==============================] - 0s 5ms/step - loss: 3.1320 - val_loss: 7472.8652
Epoch 2741/3000
9/9 [==============================] - 0s 5ms/step - loss: 2.9488 - val_loss: 7460.1460
Epoch 2742/3000
9/9 [==============================] - 0s 5ms/step - loss: 2.1966 - val_loss: 7547.9341
Epoch 2743/3000
9/9 [==============================] - 0s 5ms/step - loss: 2.4662 - val_loss: 7478.0273
Epoch 2744/3000
9/9 [==============================] - 0s 5ms/step - loss: 2.1795 - val_loss: 7567.1909
Epoch 2745/3000
9/9 [==============================] - 0s 5ms/step - loss: 2.0202 - val_loss: 7594.4531
Epoch 2746/3000
9/9 [==============================] - 0s 5ms/step - loss: 1.7390 - val_loss: 7500.9858
Epoch 2747/3000
9/9 [==============================] - 0s 5ms/step - loss: 2.4986 - val_loss: 7466.1030
Epoch 2748/3000
9/9 [==============================] - 0s 5ms/step - loss: 3.139

Epoch 2818/3000
9/9 [==============================] - 0s 5ms/step - loss: 3.9448 - val_loss: 7560.2061
Epoch 2819/3000
9/9 [==============================] - 0s 5ms/step - loss: 4.6547 - val_loss: 7548.3594
Epoch 2820/3000
9/9 [==============================] - 0s 5ms/step - loss: 4.2793 - val_loss: 7464.7246
Epoch 2821/3000
9/9 [==============================] - 0s 4ms/step - loss: 2.7456 - val_loss: 7493.9868
Epoch 2822/3000
9/9 [==============================] - 0s 4ms/step - loss: 2.5903 - val_loss: 7591.0122
Epoch 2823/3000
9/9 [==============================] - 0s 5ms/step - loss: 2.1536 - val_loss: 7424.8105
Epoch 2824/3000
9/9 [==============================] - 0s 4ms/step - loss: 2.3109 - val_loss: 7546.4780
Epoch 2825/3000
9/9 [==============================] - 0s 4ms/step - loss: 2.3887 - val_loss: 7486.9209
Epoch 2826/3000
9/9 [==============================] - 0s 5ms/step - loss: 3.2857 - val_loss: 7577.9287
Epoch 2827/3000
9/9 [==============================] - 0s 4ms/st

Epoch 2897/3000
9/9 [==============================] - 0s 5ms/step - loss: 8.6616 - val_loss: 7539.7554
Epoch 2898/3000
9/9 [==============================] - 0s 5ms/step - loss: 11.5139 - val_loss: 7719.0967
Epoch 2899/3000
9/9 [==============================] - 0s 5ms/step - loss: 23.7796 - val_loss: 7313.0483
Epoch 2900/3000
9/9 [==============================] - 0s 5ms/step - loss: 25.2031 - val_loss: 7826.8213
Epoch 2901/3000
9/9 [==============================] - 0s 5ms/step - loss: 24.8843 - val_loss: 7515.8027
Epoch 2902/3000
9/9 [==============================] - 0s 5ms/step - loss: 25.3732 - val_loss: 7595.5571
Epoch 2903/3000
9/9 [==============================] - 0s 5ms/step - loss: 17.2803 - val_loss: 7306.1328
Epoch 2904/3000
9/9 [==============================] - 0s 5ms/step - loss: 17.9939 - val_loss: 7412.8687
Epoch 2905/3000
9/9 [==============================] - 0s 5ms/step - loss: 13.5312 - val_loss: 7356.6260
Epoch 2906/3000
9/9 [==============================] - 0

9/9 [==============================] - 0s 5ms/step - loss: 0.7405 - val_loss: 7470.0386
Epoch 2976/3000
9/9 [==============================] - 0s 5ms/step - loss: 0.9208 - val_loss: 7451.4878
Epoch 2977/3000
9/9 [==============================] - 0s 5ms/step - loss: 0.8686 - val_loss: 7445.9868
Epoch 2978/3000
9/9 [==============================] - 0s 5ms/step - loss: 1.2379 - val_loss: 7544.3506
Epoch 2979/3000
9/9 [==============================] - 0s 5ms/step - loss: 1.8909 - val_loss: 7462.4277
Epoch 2980/3000
9/9 [==============================] - 0s 5ms/step - loss: 1.8365 - val_loss: 7490.2676
Epoch 2981/3000
9/9 [==============================] - 0s 5ms/step - loss: 1.1951 - val_loss: 7506.0688
Epoch 2982/3000
9/9 [==============================] - 0s 5ms/step - loss: 1.3280 - val_loss: 7489.2720
Epoch 2983/3000
9/9 [==============================] - 0s 5ms/step - loss: 1.1683 - val_loss: 7503.4312
Epoch 2984/3000
9/9 [==============================] - 0s 5ms/step - loss: 1.631

In [28]:
# 회귀 모델 평가
from sklearn.metrics import mean_squared_error
y_pred = model_reiu.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print('MSE : {0:.3f} , RMSE : {1:.3F}'.format(mse , rmse))

3/3 [==============================] - 0s 2ms/step
MSE : 9044.223 , RMSE : 95.101


In [22]:
#3. 신경망(예측) 
#3.2 회귀 모델 생성 - sigmoid 활성함수
import tensorflow as tf
model_sigmoid = tf.keras.Sequential([
    tf.keras.layers.Dense(units=52, activation='sigmoid', input_shape=(10,)),
    tf.keras.layers.Dense(units=39, activation='sigmoid'),
    tf.keras.layers.Dense(units=26, activation='sigmoid'),
    tf.keras.layers.Dense(units=1)
])
model_sigmoid.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01), loss='mse')
model_sigmoid.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 52)                572       
                                                                 
 dense_5 (Dense)             (None, 39)                2067      
                                                                 
 dense_6 (Dense)             (None, 26)                1040      
                                                                 
 dense_7 (Dense)             (None, 1)                 27        
                                                                 
Total params: 3,706
Trainable params: 3,706
Non-trainable params: 0
_________________________________________________________________


In [25]:
# 회귀 모델 학습 : epochs~신경망을 전체 한번 학습하여 나온 결과
# batch_size~train-set의 80%의 자료를 32개씩 소그룹(sub-group)으로 나누어 빠르게 학습
model_sigmoid.fit(x_train, y_train, epochs=3000, batch_size=40, validation_split=0.2)

Epoch 1/3000
8/8 [==============================] - 0s 9ms/step - loss: 24741.8027 - val_loss: 26631.5801
Epoch 2/3000
8/8 [==============================] - 0s 5ms/step - loss: 24715.2793 - val_loss: 26603.7891
Epoch 3/3000
8/8 [==============================] - 0s 5ms/step - loss: 24690.0586 - val_loss: 26576.8242
Epoch 4/3000
8/8 [==============================] - 0s 5ms/step - loss: 24664.8457 - val_loss: 26549.1680
Epoch 5/3000
8/8 [==============================] - 0s 5ms/step - loss: 24638.6348 - val_loss: 26521.1895
Epoch 6/3000
8/8 [==============================] - 0s 6ms/step - loss: 24612.2402 - val_loss: 26493.3164
Epoch 7/3000
8/8 [==============================] - 0s 6ms/step - loss: 24586.4395 - val_loss: 26465.7715
Epoch 8/3000
8/8 [==============================] - 0s 6ms/step - loss: 24560.5664 - val_loss: 26437.7246
Epoch 9/3000
8/8 [==============================] - 0s 7ms/step - loss: 24535.1387 - val_loss: 26410.5723
Epoch 10/3000
8/8 [===========================

8/8 [==============================] - 0s 5ms/step - loss: 22755.7773 - val_loss: 24495.3379
Epoch 78/3000
8/8 [==============================] - 0s 5ms/step - loss: 22725.0312 - val_loss: 24462.8555
Epoch 79/3000
8/8 [==============================] - 0s 5ms/step - loss: 22694.4785 - val_loss: 24430.4238
Epoch 80/3000
8/8 [==============================] - 0s 5ms/step - loss: 22664.9844 - val_loss: 24399.3887
Epoch 81/3000
8/8 [==============================] - 0s 5ms/step - loss: 22636.2188 - val_loss: 24368.8438
Epoch 82/3000
8/8 [==============================] - 0s 5ms/step - loss: 22607.4043 - val_loss: 24337.7832
Epoch 83/3000
8/8 [==============================] - 0s 6ms/step - loss: 22578.8340 - val_loss: 24307.3379
Epoch 84/3000
8/8 [==============================] - 0s 4ms/step - loss: 22550.4160 - val_loss: 24275.6543
Epoch 85/3000
8/8 [==============================] - 0s 5ms/step - loss: 22521.6055 - val_loss: 24245.4980
Epoch 86/3000
8/8 [==============================] 

8/8 [==============================] - 0s 5ms/step - loss: 20590.6953 - val_loss: 22161.2598
Epoch 154/3000
8/8 [==============================] - 0s 5ms/step - loss: 20561.8379 - val_loss: 22130.7715
Epoch 155/3000
8/8 [==============================] - 0s 4ms/step - loss: 20533.6523 - val_loss: 22099.9785
Epoch 156/3000
8/8 [==============================] - 0s 5ms/step - loss: 20504.6992 - val_loss: 22069.3125
Epoch 157/3000
8/8 [==============================] - 0s 5ms/step - loss: 20477.2676 - val_loss: 22038.7051
Epoch 158/3000
8/8 [==============================] - 0s 5ms/step - loss: 20448.8711 - val_loss: 22008.6836
Epoch 159/3000
8/8 [==============================] - 0s 5ms/step - loss: 20420.8672 - val_loss: 21979.1660
Epoch 160/3000
8/8 [==============================] - 0s 5ms/step - loss: 20393.5586 - val_loss: 21949.0781
Epoch 161/3000
8/8 [==============================] - 0s 5ms/step - loss: 20366.0527 - val_loss: 21918.9043
Epoch 162/3000
8/8 [=======================

Epoch 229/3000
8/8 [==============================] - 0s 5ms/step - loss: 18644.1934 - val_loss: 20054.6133
Epoch 230/3000
8/8 [==============================] - 0s 5ms/step - loss: 18620.8555 - val_loss: 20029.0352
Epoch 231/3000
8/8 [==============================] - 0s 5ms/step - loss: 18596.6992 - val_loss: 20002.6309
Epoch 232/3000
8/8 [==============================] - 0s 5ms/step - loss: 18572.6660 - val_loss: 19974.9883
Epoch 233/3000
8/8 [==============================] - 0s 6ms/step - loss: 18547.5449 - val_loss: 19948.7422
Epoch 234/3000
8/8 [==============================] - 0s 6ms/step - loss: 18524.1055 - val_loss: 19924.4590
Epoch 235/3000
8/8 [==============================] - 0s 7ms/step - loss: 18501.5742 - val_loss: 19900.0625
Epoch 236/3000
8/8 [==============================] - 0s 6ms/step - loss: 18479.8477 - val_loss: 19875.7871
Epoch 237/3000
8/8 [==============================] - 0s 6ms/step - loss: 18457.2363 - val_loss: 19851.3848
Epoch 238/3000
8/8 [========

Epoch 305/3000
8/8 [==============================] - 0s 6ms/step - loss: 16744.4082 - val_loss: 17981.7363
Epoch 306/3000
8/8 [==============================] - 0s 6ms/step - loss: 16721.2598 - val_loss: 17956.0254
Epoch 307/3000
8/8 [==============================] - 0s 5ms/step - loss: 16697.9590 - val_loss: 17930.4004
Epoch 308/3000
8/8 [==============================] - 0s 5ms/step - loss: 16674.8398 - val_loss: 17905.2520
Epoch 309/3000
8/8 [==============================] - 0s 7ms/step - loss: 16652.0254 - val_loss: 17880.5137
Epoch 310/3000
8/8 [==============================] - 0s 5ms/step - loss: 16629.0977 - val_loss: 17855.7949
Epoch 311/3000
8/8 [==============================] - 0s 6ms/step - loss: 16606.9395 - val_loss: 17831.2773
Epoch 312/3000
8/8 [==============================] - 0s 5ms/step - loss: 16584.2871 - val_loss: 17806.3145
Epoch 313/3000
8/8 [==============================] - 0s 6ms/step - loss: 16561.1133 - val_loss: 17780.3984
Epoch 314/3000
8/8 [========

...

8/8 [==============================] - 0s 5ms/step - loss: 1396.5376 - val_loss: 3564.7554
Epoch 2667/3000
8/8 [==============================] - 0s 5ms/step - loss: 1405.4337 - val_loss: 3546.2170
Epoch 2668/3000
8/8 [==============================] - 0s 5ms/step - loss: 1407.6688 - val_loss: 3527.4014
Epoch 2669/3000
8/8 [==============================] - 0s 5ms/step - loss: 1399.0519 - val_loss: 3536.5889
Epoch 2670/3000
8/8 [==============================] - 0s 5ms/step - loss: 1396.8142 - val_loss: 3546.8315
Epoch 2671/3000
8/8 [==============================] - 0s 5ms/step - loss: 1393.7924 - val_loss: 3550.2832
Epoch 2672/3000
8/8 [==============================] - 0s 5ms/step - loss: 1392.9260 - val_loss: 3552.8159
Epoch 2673/3000
8/8 [==============================] - 0s 5ms/step - loss: 1392.8893 - val_loss: 3550.4600
Epoch 2674/3000
8/8 [==============================] - 0s 5ms/step - loss: 1389.5758 - val_loss: 3543.6677
Epoch 2675/3000
8/8 [==============================] 

8/8 [==============================] - 0s 5ms/step - loss: 1337.6630 - val_loss: 3716.0906
Epoch 2743/3000
8/8 [==============================] - 0s 5ms/step - loss: 1325.4761 - val_loss: 3708.4282
Epoch 2744/3000
8/8 [==============================] - 0s 5ms/step - loss: 1357.0095 - val_loss: 3745.0571
Epoch 2745/3000
8/8 [==============================] - 0s 5ms/step - loss: 1406.3804 - val_loss: 3761.2314
Epoch 2746/3000
8/8 [==============================] - 0s 5ms/step - loss: 1355.1715 - val_loss: 3728.4211
Epoch 2747/3000
8/8 [==============================] - 0s 5ms/step - loss: 1370.3131 - val_loss: 3768.3777
Epoch 2748/3000
8/8 [==============================] - 0s 5ms/step - loss: 1367.2657 - val_loss: 3719.0916
Epoch 2749/3000
8/8 [==============================] - 0s 5ms/step - loss: 1368.2999 - val_loss: 3723.5806
Epoch 2750/3000
8/8 [==============================] - 0s 5ms/step - loss: 1438.8683 - val_loss: 3591.3542
Epoch 2751/3000
8/8 [==============================] 

8/8 [==============================] - 0s 5ms/step - loss: 1269.5171 - val_loss: 3763.8142
Epoch 2819/3000
8/8 [==============================] - 0s 5ms/step - loss: 1265.3744 - val_loss: 3755.9106
Epoch 2820/3000
8/8 [==============================] - 0s 5ms/step - loss: 1259.8721 - val_loss: 3718.3979
Epoch 2821/3000
8/8 [==============================] - 0s 5ms/step - loss: 1267.6790 - val_loss: 3697.0837
Epoch 2822/3000
8/8 [==============================] - 0s 5ms/step - loss: 1255.7106 - val_loss: 3685.2046
Epoch 2823/3000
8/8 [==============================] - 0s 5ms/step - loss: 1255.8810 - val_loss: 3692.0413
Epoch 2824/3000
8/8 [==============================] - 0s 5ms/step - loss: 1253.9878 - val_loss: 3698.6470
Epoch 2825/3000
8/8 [==============================] - 0s 5ms/step - loss: 1258.1399 - val_loss: 3723.7456
Epoch 2826/3000
8/8 [==============================] - 0s 5ms/step - loss: 1259.0071 - val_loss: 3741.7297
Epoch 2827/3000
8/8 [==============================] 

8/8 [==============================] - 0s 5ms/step - loss: 1206.4965 - val_loss: 3831.1377
Epoch 2895/3000
8/8 [==============================] - 0s 5ms/step - loss: 1239.4182 - val_loss: 3826.9041
Epoch 2896/3000
8/8 [==============================] - 0s 5ms/step - loss: 1212.0887 - val_loss: 3780.2512
Epoch 2897/3000
8/8 [==============================] - 0s 5ms/step - loss: 1199.5962 - val_loss: 3789.5344
Epoch 2898/3000
8/8 [==============================] - 0s 5ms/step - loss: 1209.1823 - val_loss: 3786.1316
Epoch 2899/3000
8/8 [==============================] - 0s 5ms/step - loss: 1189.5621 - val_loss: 3794.4128
Epoch 2900/3000
8/8 [==============================] - 0s 5ms/step - loss: 1193.4601 - val_loss: 3835.1672
Epoch 2901/3000
8/8 [==============================] - 0s 5ms/step - loss: 1213.4666 - val_loss: 3841.5098
Epoch 2902/3000
8/8 [==============================] - 0s 5ms/step - loss: 1200.9985 - val_loss: 3829.9604
Epoch 2903/3000
8/8 [==============================] 

8/8 [==============================] - 0s 5ms/step - loss: 1133.0238 - val_loss: 3729.5613
Epoch 2971/3000
8/8 [==============================] - 0s 5ms/step - loss: 1140.6896 - val_loss: 3732.5779
Epoch 2972/3000
8/8 [==============================] - 0s 5ms/step - loss: 1161.0728 - val_loss: 3694.2061
Epoch 2973/3000
8/8 [==============================] - 0s 5ms/step - loss: 1140.9192 - val_loss: 3675.3315
Epoch 2974/3000
8/8 [==============================] - 0s 5ms/step - loss: 1128.9681 - val_loss: 3680.0073
Epoch 2975/3000
8/8 [==============================] - 0s 5ms/step - loss: 1138.9237 - val_loss: 3691.9546
Epoch 2976/3000
8/8 [==============================] - 0s 5ms/step - loss: 1122.1558 - val_loss: 3674.3943
Epoch 2977/3000
8/8 [==============================] - 0s 5ms/step - loss: 1183.4615 - val_loss: 3721.9807
Epoch 2978/3000
8/8 [==============================] - 0s 5ms/step - loss: 1192.2465 - val_loss: 3651.4209
Epoch 2979/3000
8/8 [==============================] 

In [26]:
# 회귀 모델 평가
from sklearn.metrics import mean_squared_error
y_pred = model_sigmoid.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print('MSE : {0:.3f} , RMSE : {1:.3F}'.format(mse , rmse))

3/3 [==============================] - 0s 2ms/step
MSE : 4909.516 , RMSE : 70.068
